In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam



ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995


class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)


def cartpole():
    env = gym.make(ENV_NAME)
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            env.render()
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                print(f"Run: {run} exploration: {dqn_solver.exploration_rate} score: {step}")
                break
            dqn_solver.experience_replay()

cartpole()

Run: 1 exploration: 0.9416228069143757 score: 32
Run: 2 exploration: 0.8734200960253871 score: 16
Run: 3 exploration: 0.8433051360508336 score: 8
Run: 4 exploration: 0.7628626641409962 score: 21
Run: 5 exploration: 0.7183288830986236 score: 13
Run: 6 exploration: 0.6797938283326578 score: 12
Run: 7 exploration: 0.6401093727576664 score: 13
Run: 8 exploration: 0.6057704364907278 score: 12
Run: 9 exploration: 0.5790496471185967 score: 10
Run: 10 exploration: 0.5535075230322891 score: 10
Run: 11 exploration: 0.5134164023722473 score: 16
Run: 12 exploration: 0.47622912292284103 score: 16
Run: 13 exploration: 0.43732904629000013 score: 18
Run: 14 exploration: 0.4159480862733536 score: 11
Run: 15 exploration: 0.3614809303671764 score: 29
Run: 16 exploration: 0.3455358541129786 score: 10
Run: 17 exploration: 0.33195389135223546 score: 9
Run: 18 exploration: 0.3173112652388396 score: 10
Run: 19 exploration: 0.30028896908517405 score: 12
Run: 20 exploration: 0.2884855236625661 score: 9
Run: 21 

KeyboardInterrupt: 